In [ ]:
import json
from collections import UserDict

class DotNotationDict(UserDict):
    def __getattr__(self, item):
        if item in self.data:
            value = self.data[item]
            if isinstance(value, dict):
                return DotNotationDict(value)
            return value
        raise AttributeError(f"'DotNotationDict' object has no attribute '{item}'")

    def __setattr__(self, key, value):
        if key == 'data':
            super().__setattr__(key, value)
        else:
            self.data[key] = DotNotationDict(value) if isinstance(value, dict) else value

    def __delattr__(self, item):
        if item in self.data:
            del self.data[item]
        else:
            raise AttributeError(f"'DotNotationDict' object has no attribute '{item}'")

    def __getitem__(self, key):
        value = super().__getitem__(key)
        if isinstance(value, dict):
            return DotNotationDict(value)
        return value

    def __setitem__(self, key, value):
        super().__setitem__(key, DotNotationDict(value) if isinstance(value, dict) else value)

    def get(self, item, default=None):
        keys = item.split('.')
        value = self.data
        key = keys.pop(0)
        if key in value:
            
            if not isinstance(value[key], dict):
                value = value[key]
            else:
                return default
        else:
            return default
        if isinstance(value, DotNotationDict):
            return value.get('.'.join(keys), default)
        return value

    def to_dict(self):
        return {key: (value.to_dict() if isinstance(value, DotNotationDict) else value)
                for key, value in self.data.items()}

    def to_json(self):
        return json.dumps(self.to_dict())

    @staticmethod
    def from_json(json_str):
        return DotNotationDict(json.loads(json_str))


In [ ]:

# Test cases
data = DotNotationDict({'tiles': {'player': {'up': 'value_up', 'down': 'value_down'}}})
assert data.tiles.player.up == 'value_up'
assert data.tiles.player.down == 'value_down'
assert data.get('tiles.player.up') == 'value_up'
assert data.get('tiles.player.down') == 'value_down'
assert data.get('tiles.player.left') is None
data.tiles.player.left = 'value_left'
assert data.get('tiles.player.left') == 'value_left'
assert data.to_dict() == {'tiles': {'player': {'up': 'value_up', 'down': 'value_down', 'left': 'value_left'}}}
assert data.to_json() == '{"tiles": {"player": {"up": "value_up", "down": "value_down", "left": "value_left"}}}'


In [ ]:
tiles.get("player.up")

In [ ]:
tiles.player.up

In [ ]:
tiles.get('player.left')